<a href="https://colab.research.google.com/github/KillerStrike17/DeepLearning_for_NLP/blob/master/Recurrent_Neural_Networks_101/IRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a reproduction of the IRNN experiment with pixel-by-pixel sequential MNIST in "A Simple Way to Initialize Recurrent Networks of Rectified Linear Units" by Quoc V. Le, Navdeep Jaitly, Geoffrey E. Hinton

arxiv:1504.00941v2 [cs.NE] 7 Apr 2015 http://arxiv.org/pdf/1504.00941v2.pdf

Optimizer is replaced with RMSprop which yields more stable and steady improvement.

Reaches 0.93 train/test accuracy after 900 epochs (which roughly corresponds to 1687500 steps in the original paper.)

# Importing Libraries

In [0]:
!nvidia-smi

Mon Feb  3 09:26:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Calling the GODs

%tensorflow_version 1.x
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras import initializers
from keras.optimizers import RMSprop

Using TensorFlow backend.


# Creating Dataset

In [0]:
# Hyperparamter Values
batch_size = 1024
num_classes = 10
epochs = 10
hidden_units = 100
learning_rate = 1e-6
# clip_norm = 1.0

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 2s 0us/step
x_train shape: (60000, 784, 1)
60000 train samples
10000 test samples


# Model


In [0]:
model = Sequential()
model.add(SimpleRNN(hidden_units, activation='relu', input_shape=x_train.shape[1:]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])
model.summary()







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 39s 652us/step - loss: 2.3026 - acc: 0.1113 - val_loss: 2.3026 - val_acc: 0.1138
Epoch 2/10
60000/60000 [==============================] - 30s 495us/step - loss: 2.3026 - acc: 0.1125 - val_loss: 2.3026 - val_acc: 0.1136
Epoch 3/10
60000/60000 [==============================] - 30s 492us/step - loss: 2.3026 - acc: 0.1124 - val_loss: 2.3026 - val_acc: 0.1136
Epoch 4/10
60000/60000 [==============================] - 29s 491us/step - loss: 2.3026 - acc: 0.1124 - val_loss: 2.3026 - val_acc: 0.1136
Epoch 5/10
60000/60000 [==============================] - 29s 491us/step - loss: 2.3026 - acc: 0.1124 - val_loss: 2.3026 - val_acc: 0.1136
Epoch 6/10
60000/60000 [==============================] - 28s 474us/step - loss: 2.3026 - acc: 0.1124 - val_loss: 2.3026 - val_acc: 0.1136
Epoch 7/10
600

# Experiments

## Experiment1

Added batchsize,kernel Initializer and Recurrent Initilizers as mentioned in the paper.

In [0]:
#Model

print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units,kernel_initializer=initializers.RandomNormal(stddev=0.001),recurrent_initializer=initializers.Identity(gain=1.0), activation='relu', input_shape=x_train.shape[1:]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model.summary()


Evaluate IRNN...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________


In [0]:
batch_size = 2048
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 26s 432us/step - loss: 2.2930 - acc: 0.0984 - val_loss: 2.2867 - val_acc: 0.0981
Epoch 2/10
60000/60000 [==============================] - 15s 252us/step - loss: 2.2815 - acc: 0.1024 - val_loss: 2.2758 - val_acc: 0.1128
Epoch 3/10
60000/60000 [==============================] - 15s 258us/step - loss: 2.2677 - acc: 0.1362 - val_loss: 2.2585 - val_acc: 0.1627
Epoch 4/10
60000/60000 [==============================] - 15s 258us/step - loss: 2.2472 - acc: 0.1777 - val_loss: 2.2351 - val_acc: 0.1941
Epoch 5/10
60000/60000 [==============================] - 15s 255us/step - loss: 2.2215 - acc: 0.2058 - val_loss: 2.2075 - val_acc: 0.2088
Epoch 6/10
60000/60000 [==============================] - 15s 254us/step - loss: 2.1919 - acc: 0.2196 - val_loss: 2.1762 - val_acc: 0.2217
Epoch 7/10
600

## Experiment2

Added kernel Initializer and Recurrent Initilizers as mentioned in the paper and added an new layer with same config as before


In [0]:
#Model

print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units,kernel_initializer=initializers.RandomNormal(stddev=0.001),recurrent_initializer=initializers.Identity(gain=1.0), activation='relu', input_shape=x_train.shape[1:],return_sequences=True))
model.add(SimpleRNN(hidden_units,kernel_initializer=initializers.RandomNormal(stddev=0.001),recurrent_initializer=initializers.Identity(gain=1.0), activation='relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model.summary()


Evaluate IRNN...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 784, 100)          10200     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 31,310
Trainable params: 31,310
Non-trainable params: 0
_________________________________________________________________


In [0]:
batch_size = 32
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 1795s 30ms/step - loss: 1.8914 - acc: 0.3013 - val_loss: 1.6781 - val_acc: 0.3766
Epoch 2/10
60000/60000 [==============================] - 1757s 29ms/step - loss: 1.5562 - acc: 0.4374 - val_loss: 1.5529 - val_acc: 0.4501
Epoch 3/10
60000/60000 [==============================] - 1759s 29ms/step - loss: 1.3904 - acc: 0.5028 - val_loss: 1.2969 - val_acc: 0.5611
Epoch 4/10
60000/60000 [==============================] - 1766s 29ms/step - loss: 1.2920 - acc: 0.5385 - val_loss: 1.2222 - val_acc: 0.5684
Epoch 5/10
60000/60000 [==============================] - 1833s 31ms/step - loss: 1.2453 - acc: 0.5536 - val_loss: 1.3381 - val_acc: 0.4934
Epoch 6/10
60000/60000 [==============================] - 1855s 31ms/step - loss: 1.2088 - acc: 0.5676 - val_loss: 1.2047 - val_acc: 0.5564
Epoch 7/

## Experiment3

Added kernel Initializer and Recurrent Initilizers as mentioned in the paper and added an new layer with random values


In [0]:
#Model

print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units,kernel_initializer=initializers.RandomNormal(stddev=0.001),recurrent_initializer=initializers.Identity(gain=1.0), activation='relu', input_shape=x_train.shape[1:],return_sequences=True))
model.add(SimpleRNN(hidden_units,activation='relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model.summary()


Evaluate IRNN...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 784, 100)          10200     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 31,310
Trainable params: 31,310
Non-trainable params: 0
_________________________________________________________________


In [0]:
batch_size = 32
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 2176s 36ms/step - loss: 2.1610 - acc: 0.2058 - val_loss: 2.0181 - val_acc: 0.2388
Epoch 2/10
60000/60000 [==============================] - 2139s 36ms/step - loss: 2.0124 - acc: 0.2415 - val_loss: 2.0002 - val_acc: 0.2478
Epoch 3/10
60000/60000 [==============================] - 2144s 36ms/step - loss: 1.9797 - acc: 0.2555 - val_loss: 1.9784 - val_acc: 0.2580
Epoch 4/10
60000/60000 [==============================] - 2149s 36ms/step - loss: 1.9422 - acc: 0.2776 - val_loss: 1.9068 - val_acc: 0.3168
Epoch 5/10
60000/60000 [==============================] - 2120s 35ms/step - loss: 1.8890 - acc: 0.3044 - val_loss: 1.8428 - val_acc: 0.3198
Epoch 6/10
60000/60000 [==============================] - 2120s 35ms/step - loss: 1.8184 - acc: 0.3355 - val_loss: 1.7512 - val_acc: 0.3688
Epoch 7/